In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, zero_one_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import os
os.chdir('..')  # Go up one directory level
import tensorflow
import tensorflow as tf
from tensorflow import keras

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Add, InputSpec, Input, BatchNormalization, Dropout,Conv2D, Conv2DTranspose,AveragePooling2D, MaxPooling2D,Dense, Flatten, Reshape, Concatenate, Embedding, Lambda

import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from TAABLM_mechanism import GlobalContextLayer, AttentionMechanism, custom_objects

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.initializers import RandomNormal

!pwd

/var/folders/td/w85k3fwn2nn5xffjktw34x_m0000gn/T/ipykernel_85070/4096949599.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


/Users/tuongdoanvanngoc/Documents/Y3T3/378/AMS-Project/callai2/MyProject


In [2]:
from TAABLM_mechanism import GlobalContextLayer, AttentionMechanism, custom_objects
from Preprocessing_Data import load_and_preprocess_data, preprocess_texts, load_glove_embeddings

In [3]:
# datapath = data_dir + 'CallAIDataCorpus2.csv'
datapath = 'data/Account_Verification_subsection.csv'

data,texts, unique_labels = load_and_preprocess_data(datapath)

In [4]:
# Preprocess texts
padded_sequences, tokenizer, X_tfidf, word_index, index_word = preprocess_texts(texts)

# Load GloVe embeddings
embedding_dim = 50
vocab_size = 10000
max_seq_len=100

embedding_matrix = load_glove_embeddings(vocab_size, embedding_dim, word_index)

In [5]:
# Assuming data['Text'], data['Category'], and tf_idf_tensor are available
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, data['Personal Information'], test_size=0.2, random_state=42
)

# Split tf_idf_tensor separately to align with X_train and X_test
X_tfidf_train, X_tfidf_test = train_test_split(
    X_tfidf, test_size=0.2, random_state=42
)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Specify the directory containing the saved model files
dir_name = 'TAABLM_model'

# 1. Load the model architecture from the saved JSON file
with open(os.path.join(dir_name, 'TAABLM_Section_architecture.json'), 'r') as json_file:
    model_json = json_file.read()

# Recreate the model from the JSON string
model = model_from_json(model_json)


# 3. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

# 4. Define EarlyStopping callback to avoid overfitting
early_stopping = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

# Train the model
history = model.fit(
    [X_train, X_tfidf_train],  # Training inputs
    y_train,                   # Training labels
    validation_data=([X_test, X_tfidf_test], y_test),  # Validation inputs and labels
    epochs=20,                 # Number of epochs
    batch_size=16,             # Batch size
    callbacks=[early_stopping]  # Early stopping callback
)

2024-12-09 23:40:20.971660: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-09 23:40:20.971811: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-09 23:40:20.971844: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-09 23:40:20.971952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-09 23:40:20.971984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20


/opt/miniconda3/envs/gpu/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_sentence', 'input_tfidf']. Received: the structure of inputs=('*', '*')
  warnings.warn(
2024-12-09 23:40:21.802455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 341ms/step - accuracy: 0.3236 - loss: 3.3779 - val_accuracy: 0.5106 - val_loss: 2.6923
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6740 - loss: 2.0976 - val_accuracy: 0.5426 - val_loss: 2.0470
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.7130 - loss: 1.4150 - val_accuracy: 0.5638 - val_loss: 1.6429
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8110 - loss: 1.0227 - val_accuracy: 0.5532 - val_loss: 1.4027
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8587 - loss: 0.8092 - val_accuracy: 0.6064 - val_loss: 1.2209
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8384 - loss: 0.6929 - val_accuracy: 0.6809 - val_loss: 1.0688
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8719 - loss: 0.6060 - val_accuracy: 0.6702 - val_loss: 0.9815
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9228 - loss: 0.4663 - val_accuracy: 0.7234 - val_loss: 

In [7]:
# Change the directory
new_directory = "TAABLM_model"
os.chdir(new_directory)

# Specify the directory to create
dir_name = 'Account_Verification'  # Replace with your desired path

# Create the directory
os.makedirs(dir_name, exist_ok=True)
model.save(os.path.join(dir_name, 'AV_Information.h5'))